In [5]:
import requests
from urllib.request import urlopen
from urllib.parse import quote_plus
from bs4 import BeautifulSoup
import re

import pandas as pd
from pandas.io.json import json_normalize

import datetime
import time

import warnings
warnings.filterwarnings(action='ignore')

In [3]:
url = 'https://www.op.gg/summoner/champions/userName='

# 1. 소환사명

In [4]:
ch_to_ms_df = pd.read_csv('./ch_to_ms.csv')
ch_to_ms_df.drop(columns=['Unnamed: 0'], inplace=True)
ch_to_ms_df.head()

,summonerId,summonerName,leaguePoints,rank,wins,losses,veteran,inactive,freshBlood,hotStreak,accountId,puuid,profileIconId,revisionDate,summonerLevel
0,2OMUVkdEFjCQ-X9cnKnVzutigIzx39Mo62QlK5qcFIW2E5g,T1 Zeus,1143,I,459,397,True,False,False,False,d8iLopH4-JkPo4S47OtjC_VqjI3QRR1KvNs4UqmVO5V9kCA,ODWTUNHDMtK9C0h1_ObdjJ6ZDkij4DJNAAtWMrwiIQ-TyS...,5040.0,1.629800e+12,367.0
1,_iG58wg8kGfppZxQQITHQOvt8IvFVZzfSvFX88TrCcMrjp...,tan hua yi xian,977,I,877,807,False,False,False,False,3KmoGjA5MmatwKUPvhZVlYXtmIxx4TIs7CaLCUs3RwbF4k...,hcF9febiHTR-26ft4VinC0wIAIoHr2lpP0ClGOxOCZ5zAl...,7.0,1.630063e+12,152.0
2,xy686tdArwnwtqijpMWjZAYtzM45g3xTZJ1ncI4PeaD18Os,BRO Loopy,805,I,757,705,False,False,True,False,vkmZnwY1b50o7XtwItxtDit2T4HYQ36aeFPY5RRwY28Vyg...,D0RYGxX15DItJnF8nsu4ushlBuFRLLRzUZypMtY9pV8JdH...,4802.0,1.630074e+12,298.0
3,Jg00lT8Ynrkiop9yUW-yWgB0j95COw7wao6_w5pJnuJusOs,Gen G Bdd,1011,I,224,171,True,False,False,False,5PYA6Cqr8LQOX6F-Mt63cU-mlab1d0vJPO6euyCaLPqMAyk,ftmmwwL7XStkrgAlIk7QAotgJseK01_B2RNZEHEyOt0sbA...,7.0,1.630063e+12,208.0
4,2F6koT1BM5kSxA6y7E_VL--jS3XZ-3O8vCNoHayVUimOjB...,wu guan feng yue,1320,I,727,647,True,False,False,False,Hdoz8u8Oo6V7R7K6ZnowMdiXyp7OX25QWLiaIrdbOCWJ45...,jC2GVl9V2hssOLVNnWAjrqGVIbt8mlW3GyPHW68FH2Clvv...,5034.0,1.630056e+12,204.0


# 2. 소환사별 챔피언 정보 정리

In [6]:
regex = {}

regex['Champion'] = re.compile(r'\s*([\w.&]*)\s*')
regex['Played'] = re.compile(r'\s*(\d*W)*\s*(\d*L)*\s*(\d*%)+\s*')
regex['KDA'] = re.compile(r'\s*([\d.]*)[ /\s]*([\d.]*)[ /\s]*([\d.]*)[ /\s]*([\d.:]*)\s*')
regex['Gold'] = re.compile(r'\s*([\d,]*)\s*')
regex['CS'] = re.compile(r'\s*([\d.]*)[\(\s]*([\d.]*)\)\s*')
regex['Max Kills'] = re.compile(r'\s*([\d]*)\s*')
regex['Max Death'] = re.compile(r'\s*([\d]*)\s*')
regex['Average Damage Dealt'] = re.compile(r'\s*([\d,]*)\s*')
regex['Average Damage Taken'] = re.compile(r'\s*([\d,]*)\s*')
regex['Double Kill'] = re.compile(r'\s*([\d]*)\s*')
regex['Triple Kill'] = re.compile(r'\s*([\d]*)\s*')
regex['Quadra Kill'] = re.compile(r'\s*([\d]*)\s*')
regex['Penta Kill'] = re.compile(r'\s*([\d]*)\s*')

## 소환사별 챔피언 데이터 from OPGG

In [8]:
columnlist = ['#', 'Champion', 'Win', 'Loss', 'Win Rate', 'K', 'D', 'A', 'KDA', 'Gold', 'CS', 'CS/min', 
               'Max Kills', 'Max Deaths', 'Average Damage Dealt', 'Average Damage Taken',
              'Double Kill', 'Triple Kill', 'Quadra Kill', 'Penta Kill']
DF = pd.DataFrame(columns=['summonerName']+columnlist[1:])

In [13]:
j = 0
N = len(ch_to_ms_df.summonerName.values)

while j < N:
    try:    
        sn = ch_to_ms_df.summonerName.values[j]
        sn_noblank = re.sub(r'[\s]*', '', sn)
        encode = quote_plus(sn_noblank)

        html = urlopen(url+encode)
        bsObject = BeautifulSoup(html, 'html.parser')
        contents = bsObject.select('div > table > tbody > tr')

        reg = re.compile(r'[\d\w.,:&\' ]+')

        df = pd.DataFrame(columns=columnlist)
        dfcontent = []

        for con in contents:
            tds = con.find_all('td')
            del tds[1]

            for i, td in enumerate(tds):
                text = td.text
                texts = reg.findall(text)

                if i == 2:
                    played_group = regex['Played'].search(text)

                    if played_group.group(1) == None:
                        dfcontent = dfcontent + ['0W'] + texts
                    elif played_group.group(2) == None:
                        dfcontent = dfcontent + [texts[0]] + ['0L'] + [texts[1]]
                    else:
                        dfcontent = dfcontent + texts

                else:
        #         dfcontent.append(text)
                    dfcontent = dfcontent + texts

            if len(dfcontent) != len(columnlist):
                for i in range(len(columnlist)-len(dfcontent)):
                    dfcontent.append(0)

            ser = pd.Series(dfcontent, index=columnlist)
            df = df.append(ser, ignore_index=True)

            dfcontent = []

        # df = pd.DataFrame()
        # df = df.append(alldfcontents)
        df['Win'] = df['Win'].apply(lambda x: x[:-1])
        df['Loss'] = df['Loss'].apply(lambda x: x[:-1])
    
    except Exception as e:
        print('{0}   {1:5d}: {2}'.format(datetime.datetime.now().strftime("%H:%M:%S"), j, e))
        continue

    df.drop(columns=['#'], inplace=True)
    df['summonerName'] = sn
    DF = DF.append(df)
    
    if j % 100 == 0:
        print('{0}   {1:5d}: {2}'.format(datetime.datetime.now().strftime("%H:%M:%S"), j, sn))
    j += 1
    time.sleep(1)

09:49:33       0: T1 Zeus
09:51:54     100: BRO Yaharong
09:54:14     200: RiHoo
09:56:34     300: 떡잎마을 시민1
09:58:54     400: 대충살지마인생
10:01:14     500: wwwwdddddd
10:03:34     600: 왕 흠
10:05:54     700: DWG BeryL
10:08:14     800: XUBINGJIE
10:10:33     900: AfreecaTV 김따코
10:12:52    1000: JGKING
10:15:11    1100: OOO222
10:17:29    1200: Twitch 민뱅
10:19:51    1300: 행복해써
10:22:10    1400: Ga D KR
10:24:28    1500: 중국카사딘장인
10:26:49    1600: 유미충이
10:29:11    1700: 롤은 무엇인가
10:31:32    1800: qqqqwwweer
10:33:56    1900: SIave
10:36:19    2000: JackeyLove
10:38:41    2100: 마장이에요
10:41:04    2200: 불꽃성훈
10:43:26    2300: trend play
10:45:51    2400: Kishimiyu0213
10:48:14    2500: themit12
10:50:36    2600: 버스절실한사람
10:52:57    2700: JiaoQueen
10:55:23    2800: Gen G Mulgae
10:57:47    2900: 152535455
11:00:09    3000: 서 쿤
11:02:38    3100: 전투노예1호
11:05:00    3200: 찡긋수원
11:07:23    3300: 이케시아식
11:09:46    3400: 나는노력해봤어
11:12:08    3500: 아가 승현
11:14:33    3600: 뒤질게
11:16:57    3700: 머하는거지이게
11:

In [14]:
DF.to_csv('./data/opgg_ch_to_ms.csv')

## 소환사별 챔피언 데이터 정리

In [15]:
opgg_df = pd.read_csv('./data/opgg_ch_to_ms.csv')
opgg_df.drop(columns=['Unnamed: 0'], inplace=True)
opgg_df = opgg_df[['summonerName', 'Champion', 'Win', 'Loss', 'Win Rate', 'KDA', 'K', 'D', 'A', 
                  'CS', 'CS/min', 'Gold', 'Max Kills', 'Max Deaths', 'Average Damage Dealt', 'Average Damage Taken', 
                   'Double Kill', 'Triple Kill', 'Quadra Kill', 'Penta Kill']]
opgg_df.head()

,summonerName,Champion,Win,Loss,Win Rate,KDA,K,D,A,CS,CS/min,Gold,Max Kills,Max Deaths,Average Damage Dealt,Average Damage Taken,Double Kill,Triple Kill,Quadra Kill,Penta Kill
0,T1 Zeus,Jayce,66,65,50,1.47:1,4.9,6.3,4.4,181.2,7.9,"10,378",15,14,"128,825","16,420",49,8,2,0
1,T1 Zeus,Lucian,41,41,50,1.87:1,6.6,6.1,4.8,194.2,8.2,"11,431",24,14,"137,643","16,651",54,6,1,0
2,T1 Zeus,Lee Sin,39,40,49,2.12:1,6.1,5.5,5.6,172.8,7.5,"10,733",18,12,"120,203","24,710",52,7,0,0
3,T1 Zeus,Irelia,49,21,70,1.92:1,6.3,5.6,4.5,193.1,8.5,"11,049",15,16,"132,390","24,109",43,9,3,1
4,T1 Zeus,Gwen,34,31,52,1.61:1,4.6,5.8,4.7,173.9,7.5,"10,034",12,14,"109,781","23,104",32,3,0,0


In [16]:
opgg_df['Gold'] = opgg_df['Gold'].apply(lambda x: int(x.replace(',', '')))
opgg_df['Average Damage Dealt'] = opgg_df['Average Damage Dealt'].apply(lambda x: int(x.replace(',', '')))
opgg_df['Average Damage Taken'] = opgg_df['Average Damage Taken'].apply(lambda x: int(x.replace(',', '')))
opgg_df['Double Kill'] = opgg_df['Double Kill'].apply(lambda x: str(x))
opgg_df['Double Kill'] = opgg_df['Double Kill'].apply(lambda x: int(x.replace(',', '')))
def cleanse_kda(x):
    if x == 'Perfect':
        return 100.0
    else:
        return float(x[:x.find(':')])
    
opgg_df['KDA'] = opgg_df['KDA'].apply(lambda x: cleanse_kda(x))
opgg_df['Games'] = opgg_df['Win'] + opgg_df['Loss']

In [17]:
opgg_df = opgg_df[['summonerName', 'Champion', 'Win', 'Loss', 'Games', 'Win Rate', 'KDA', 'K', 'D', 'A', 
                  'CS', 'CS/min', 'Gold', 'Max Kills', 'Max Deaths', 'Average Damage Dealt', 'Average Damage Taken', 
                   'Double Kill', 'Triple Kill', 'Quadra Kill', 'Penta Kill']]
opgg_df.to_csv('./data/opgg_ch_to_ms_cleansed.csv')

# 3. 챔피언 ID 매칭

In [19]:
opgg_df = pd.read_csv('./data/opgg_ch_to_ms_cleansed.csv')
opgg_df.drop(columns=['Unnamed: 0'], inplace=True)
opgg_df.head()

,summonerName,Champion,Win,Loss,Games,Win Rate,KDA,K,D,A,...,CS/min,Gold,Max Kills,Max Deaths,Average Damage Dealt,Average Damage Taken,Double Kill,Triple Kill,Quadra Kill,Penta Kill
0,T1 Zeus,Jayce,66,65,131,50,1.47,4.9,6.3,4.4,...,7.9,10378,15,14,128825,16420,49,8,2,0
1,T1 Zeus,Lucian,41,41,82,50,1.87,6.6,6.1,4.8,...,8.2,11431,24,14,137643,16651,54,6,1,0
2,T1 Zeus,Lee Sin,39,40,79,49,2.12,6.1,5.5,5.6,...,7.5,10733,18,12,120203,24710,52,7,0,0
3,T1 Zeus,Irelia,49,21,70,70,1.92,6.3,5.6,4.5,...,8.5,11049,15,16,132390,24109,43,9,3,1
4,T1 Zeus,Gwen,34,31,65,52,1.61,4.6,5.8,4.7,...,7.5,10034,12,14,109781,23104,32,3,0,0


In [20]:
VERSION = '11.16.1'

#champion info load
champ_req = requests.get('http://ddragon.leagueoflegends.com/cdn/' + VERSION + '/data/en_US/champion.json')
champ_ls = list(champ_req.json()['data'].keys())
champ_df = pd.DataFrame()

for i in range(len(champ_ls)):
    pre_df = json_normalize(champ_req.json()['data'][champ_ls[i]])
    champ_df = champ_df.append(pre_df)
    
#column 정리
champ_df.columns = champ_df.columns.str.strip().str.replace('info.', '')
champ_df.columns = champ_df.columns.str.strip().str.replace('stats.', '')

champ_df['tag1'] = champ_df['tags'].apply(lambda x: x[0])
champ_df['tag2'] = champ_df['tags'].apply(lambda x: x[1] if len(x) == 2 else None)

champ_df.drop(columns=['tags', 'blurb', 'image.full', 'image.group', 'image.h', 'image.sprite', 'image.w', 'image.x', 'image.y', 'title'], inplace=True)
champ_df.to_csv(f'./data/champ_data_{VERSION}.csv')

champ_key_df = champ_df[['name','key']]
champ_key_df.rename(columns={'name':'Champion',
                            'key':'championId'},
                   inplace=True)
champ_key_df.head()

,Champion,championId
0,Aatrox,266
0,Ahri,103
0,Akali,84
0,Akshan,166
0,Alistar,12


In [21]:
opgg_df = pd.merge(opgg_df, champ_key_df, on='Champion', how='left')
opgg_df.to_csv('./data/opgg_ch_to_ms_cleansed.csv')
opgg_df.head()

,summonerName,Champion,Win,Loss,Games,Win Rate,KDA,K,D,A,...,Gold,Max Kills,Max Deaths,Average Damage Dealt,Average Damage Taken,Double Kill,Triple Kill,Quadra Kill,Penta Kill,championId
0,T1 Zeus,Jayce,66,65,131,50,1.47,4.9,6.3,4.4,...,10378,15,14,128825,16420,49,8,2,0,126
1,T1 Zeus,Lucian,41,41,82,50,1.87,6.6,6.1,4.8,...,11431,24,14,137643,16651,54,6,1,0,236
2,T1 Zeus,Lee Sin,39,40,79,49,2.12,6.1,5.5,5.6,...,10733,18,12,120203,24710,52,7,0,0,64
3,T1 Zeus,Irelia,49,21,70,70,1.92,6.3,5.6,4.5,...,11049,15,16,132390,24109,43,9,3,1,39
4,T1 Zeus,Gwen,34,31,65,52,1.61,4.6,5.8,4.7,...,10034,12,14,109781,23104,32,3,0,0,887


# 4. MATCH 데이터와 소환사 데이터 merge

## match 데이터

In [22]:
matches_df = pd.read_csv('./data/matches.csv')
matches_df.drop(columns=['Unnamed: 0'], inplace=True)

print('전체 데이터:         ', len(matches_df))

classic_matches_df = matches_df.loc[matches_df['gameMode'] == 'CLASSIC'] #소환사의 협곡 데이터만
print('소환사의 협곡 데이터: ', len(classic_matches_df))

classic_matches_df.tail()

전체 데이터:          22028
소환사의 협곡 데이터:  17140


,gameId,platformId,gameCreation,gameDuration,queueId,mapId,seasonId,gameVersion,gameMode,gameType,...,team2_4_position,team2_5_championId,team2_5_spell1Id,team2_5_spell2Id,team2_5_accountId,team2_5_summonerName,team2_5_summonerId,team2_5_lane,team2_5_role,team2_5_position
22023,5333639824,KR,1626624841496,1593,420,11,13,11.14.385.9967,CLASSIC,MATCHED_GAME,...,ADC,236,4,12,FQUXCmwOAevOunI07woOgFZclpBO2nQwio3jKArmME11Ju...,phqh,cN6xykXcH9GmkaFly8JBuVz_dKCkJXRcKtjehZXZRXwzP4Zx,MIDDLE,SOLO,MID
22024,5322629780,KR,1626193694778,1509,420,11,13,11.14.385.9967,CLASSIC,MATCHED_GAME,...,SUP,517,12,4,adM8whEGwxRScnsBdiuNNOnV6-2AZ_cbO3caG5nqaoR6WC...,buwandian thx,Tg_hHN4GQfIihb5E3ZtiooolN9k_ZvLB6u5zlVxr8kbwjj...,TOP,DUO,TOP
22025,5422768789,KR,1630243256568,1950,430,11,13,11.17.394.4489,CLASSIC,MATCHED_GAME,...,JUG,31,12,4,xR7YiNhn-q4kzBfuC6798kavx3G8XaX2nL1i-FAjFtAZsn...,Caudatus,qPuflKQe6_leE3o8OoZEvaLDBdUW-YwSKEmPJbrlw95qCbw,MIDDLE,SOLO,MID
22026,5371912854,KR,1628157259300,2003,420,11,13,11.15.389.2308,CLASSIC,MATCHED_GAME,...,JUG,82,12,4,_X69VbcYUkjJSF1TaCQozN6hwC0mv3Eln_uJFslz07qBdGg,black kitty2000,DT0zUeX3RvxA9La4lHbcSERa9VcwlAb4mvAlNXbw-rEPCdk,MIDDLE,SOLO,MID
22027,5411234455,KR,1629767219797,1535,420,11,13,11.16.390.1945,CLASSIC,MATCHED_GAME,...,SUP,22,4,7,MkHHaKwl2_tGVq0egU5FSBCf60D1kud_xoguc6xAac1Cah...,VANTA,zGtMDD0mTEg6FZP3IdCuPXtYsYFeYYcduHh-qABEHTr75a...,BOTTOM,DUO_CARRY,ADC


## 소환사 ID - 소환사 이름 매칭

In [25]:
# dictionary{ summonerId : summonerName } 만들기
ch_to_ms_df = pd.read_csv('./data/ch_to_ms.csv')
ch_to_ms_df.drop(columns=['Unnamed: 0'], inplace=True)
print('challenger to master: ', len(ch_to_ms_df))

summonerIds = ch_to_ms_df.summonerId.values
summonerNames = ch_to_ms_df.summonerName.values
summonerIds_dict = dict(zip(summonerIds, summonerNames))

# summonerId에 따라 summonerName 돌려주는 함수
def get_summonerName(summonerId):
    if summonerId in summonerIds_dict:
        return summonerIds_dict[summonerId]
    else:
        return None

# summonerId에 맞게 summonerName 수정
for team in [1,2]:
    for summoner in range(1,6):
        classic_matches_df['team{0}_{1}_summonerName'.format(team, summoner)] = \
                classic_matches_df['team{0}_{1}_summonerId'.format(team, summoner)].map(lambda x: get_summonerName(x))
        
        
# nan 값 처리
classic_matches_df.dropna(inplace=True)


# summonerName:championId 형태로 key 설정
for team in [1,2]:
    for summoner in range(1,6):
        classic_matches_df['team{0}_{1}_key'.format(team, summoner)] = classic_matches_df['team{0}_{1}_summonerName'.format(team, summoner)] + \
        ':' +  classic_matches_df['team{0}_{1}_championId'.format(team, summoner)].apply(lambda x: str(x))
        
        
print('데이터 수: ', len(classic_matches_df))
classic_matches_df.head()
classic_matches_df.to_csv('./data/classic_matches.csv')

challenger to master:  4182
데이터 수:  781


## match 데이터와 소환사별 챔피언 데이터 merge

In [27]:
opgg_df = pd.read_csv('./data/opgg_ch_to_ms_cleansed.csv')
opgg_df.drop(columns=['Unnamed: 0'], inplace=True)


# 소환사이름:챔피언아이디 로 key 설정
opgg_df['key'] = opgg_df['summonerName'] + ':' + opgg_df['championId'].apply(lambda x: str(x))
opgg_df.head()

,summonerName,Champion,Win,Loss,Games,Win Rate,KDA,K,D,A,...,Max Kills,Max Deaths,Average Damage Dealt,Average Damage Taken,Double Kill,Triple Kill,Quadra Kill,Penta Kill,championId,key
0,T1 Zeus,Jayce,66,65,131,50,1.47,4.9,6.3,4.4,...,15,14,128825,16420,49,8,2,0,126,T1 Zeus:126
1,T1 Zeus,Lucian,41,41,82,50,1.87,6.6,6.1,4.8,...,24,14,137643,16651,54,6,1,0,236,T1 Zeus:236
2,T1 Zeus,Lee Sin,39,40,79,49,2.12,6.1,5.5,5.6,...,18,12,120203,24710,52,7,0,0,64,T1 Zeus:64
3,T1 Zeus,Irelia,49,21,70,70,1.92,6.3,5.6,4.5,...,15,16,132390,24109,43,9,3,1,39,T1 Zeus:39
4,T1 Zeus,Gwen,34,31,65,52,1.61,4.6,5.8,4.7,...,12,14,109781,23104,32,3,0,0,887,T1 Zeus:887


In [28]:
classic_matches_df = pd.read_csv('./data/classic_matches.csv')
classic_matches_df.drop(columns=['Unnamed: 0'], inplace=True)
classic_matches_df.head()

,gameId,platformId,gameCreation,gameDuration,queueId,mapId,seasonId,gameVersion,gameMode,gameType,...,team1_1_key,team1_2_key,team1_3_key,team1_4_key,team1_5_key,team2_1_key,team2_2_key,team2_3_key,team2_4_key,team2_5_key
0,5375524906,KR,1628309892584,1877,420,11,13,11.15.389.2308,CLASSIC,MATCHED_GAME,...,그브그브그브그브:104,no gank gg zz:119,민두홍 조각1:164,zxsazx:80,asdasdafasf0802:68,Kylian Mp3:81,cxybaby:136,tramkam:79,Fox B jingjing:64,죽림초:114
1,5384962130,KR,1628677901480,2212,420,11,13,11.16.390.1945,CLASSIC,MATCHED_GAME,...,갱값하는미드:7,kaixin1dian:111,Saturday Mornlng:517,의연하게대처하다:360,롤와일더YouTube:421,ricericerice:887,DangerousBoy ady:497,FA OneZang:22,삼생 만물:246,호잇이:79
2,5417468012,KR,1630052640044,1456,420,11,13,11.17.393.607,CLASSIC,MATCHED_GAME,...,ka ka luo te:59,wo yao likai le:39,낙타가 낙타낫따:40,미안해 말렸어:81,리리컬 매지컬:517,RAzhao369840790:360,떠있어 정글 위:41,NGU Finale:777,더불어다만진당:89,Karthus Jug:245
3,5387583710,KR,1628784873892,1718,420,11,13,11.16.390.1945,CLASSIC,MATCHED_GAME,...,a little story:131,해바라기cl:164,shenzhenshaoye:119,snzll:89,By 불비:234,어리고싶다:114,T1 Cuzz:121,AF Le5:429,Honpi:497,DK Pullbae:134
4,5411701012,KR,1629783699985,1768,420,11,13,11.16.390.1945,CLASSIC,MATCHED_GAME,...,JUGEGO:141,fakelove777:164,그래서니티어뭐임:245,chaojibaobeijojo:81,길을찾고있는사람:235,나는쓰레기1:497,총을 다루는 소녀:777,듀오강의코코팀:115,Canyon11:121,hayquatroiluon:58


In [29]:
summoner_champion_properties = ['Games', 'Win Rate', 'KDA']

for team in [1,2]:
    for summoner in range(1,6):
        temp_opgg_df = opgg_df[['key'] + summoner_champion_properties]
        temp_opgg_df.columns = temp_opgg_df.columns.map(lambda x: 'team{0}_{1}_'.format(team, summoner) + x)

        matches_for_train_df = pd.merge(classic_matches_df, temp_opgg_df, 
                                left_on='team{0}_{1}_key'.format(team, summoner), 
                                right_on='team{0}_{1}_key'.format(team, summoner), 
                                how='left')
        
# nan 값 처리
matches_for_train_df.dropna(inplace=True)
print('데이터 수: ', len(matches_for_train_df))
matches_for_train_df.to_csv('./data/matches_for_train.csv')
matches_for_train_df.head()

데이터 수:  652


,gameId,platformId,gameCreation,gameDuration,queueId,mapId,seasonId,gameVersion,gameMode,gameType,...,team1_4_key,team1_5_key,team2_1_key,team2_2_key,team2_3_key,team2_4_key,team2_5_key,team2_5_Games,team2_5_Win Rate,team2_5_KDA
0,5375524906,KR,1628309892584,1877,420,11,13,11.15.389.2308,CLASSIC,MATCHED_GAME,...,zxsazx:80,asdasdafasf0802:68,Kylian Mp3:81,cxybaby:136,tramkam:79,Fox B jingjing:64,죽림초:114,556.0,54.0,1.97
1,5384962130,KR,1628677901480,2212,420,11,13,11.16.390.1945,CLASSIC,MATCHED_GAME,...,의연하게대처하다:360,롤와일더YouTube:421,ricericerice:887,DangerousBoy ady:497,FA OneZang:22,삼생 만물:246,호잇이:79,95.0,53.0,2.20
2,5417468012,KR,1630052640044,1456,420,11,13,11.17.393.607,CLASSIC,MATCHED_GAME,...,미안해 말렸어:81,리리컬 매지컬:517,RAzhao369840790:360,떠있어 정글 위:41,NGU Finale:777,더불어다만진당:89,Karthus Jug:245,9.0,33.0,2.13
3,5387583710,KR,1628784873892,1718,420,11,13,11.16.390.1945,CLASSIC,MATCHED_GAME,...,snzll:89,By 불비:234,어리고싶다:114,T1 Cuzz:121,AF Le5:429,Honpi:497,DK Pullbae:134,51.0,51.0,1.97
4,5411701012,KR,1629783699985,1768,420,11,13,11.16.390.1945,CLASSIC,MATCHED_GAME,...,chaojibaobeijojo:81,길을찾고있는사람:235,나는쓰레기1:497,총을 다루는 소녀:777,듀오강의코코팀:115,Canyon11:121,hayquatroiluon:58,4.0,50.0,1.06
